Test notebook for pulling API data

Testing using EtherScan - getting historical gas prices: https://docs.etherscan.io/api-endpoints/gas-tracker#get-daily-average-gas-limit

In [1]:
# Import Packages
import requests
import json
import pandas as pd


Ethereum Price - Simple Method

In [34]:
VALID_ACTION = {'ethsupply2', 'ethprice'}

def get_ethstats(action):
    if action not in VALID_ACTION:
        raise ValueError("Action must be one of %r." % VALID_ACTION)
    url = f"https://api.etherscan.io/api?module=stats&action={action}&apikey={api_key}"
    #print(url)
    res = requests.get(url)
    return res.json()




In [42]:
SupplyData = pd.json_normalize(get_ethstats('ethsupply2')['result'])
print(SupplyData)

PriceData = pd.json_normalize(get_ethstats('ethprice')['result'])
print(PriceData)



                     EthSupply                Eth2Staking  \
0  122373866217800000000000000  1187683717196251000000000   

                   BurntFees             WithdrawnTotal  
0  3243849089305856770977483  2484951743300237000000000  
    ethbtc ethbtc_timestamp   ethusd ethusd_timestamp
0  0.06679       1683716709  1844.27       1683716703
